In [67]:
import sys
import os
import json
from datetime import datetime 
import numpy as np
import pandas as pd
params={'startdate':'2018-07-10','year':25,'size':99.5,'weight':1.5,'averagetime':3.4,"scenario":"lm_model","type":"month"
        ,"plant":"solar", "contruction":150000000,"investment":180000000,"othercost":25000000,"principal":120000000,"interest":0.05,
        "unredeemed":12321321123,"duration":12,"repayment_method":"cpm","repayment_term":"y","interest_repayment_term":"y","finance_startdate":'2018-06-13'}
def money_trim(array):
    result=round(array,-1)
    result=result.astype(int)
    return result

#원금균등상환(Constant Amortization Mortgage)
def CAM_calc(startdate,duration,principal,interest,term,interest_term):
    start=datetime.strptime(startdate,'%Y-%m-%d')
    if(interest_term=="y"):
        amortization=pd.DataFrame(index=pd.date_range(start,periods=params["duration"],freq="y").shift(n=-(start.month+1),freq="m").shift(n=start.day,freq="d"))
        amortization["principal"]=0; amortization["remained_principal"]=0;amortization["interest"]=0;amortization["payback"]=0;
        amortization["remained_principal"][0]=principal;
        amortization["principal"]=principal/duration;amortization.principal[0]=0
        for i in range(1, amortization.shape[0]):
            amortization.remained_principal.iloc[i]=amortization.remained_principal.iloc[i-1]-amortization.principal.iloc[i];
            amortization.interest[i]=amortization.remained_principal[i-1]*interest
    if(interest_term=="q"):
        amortization=pd.DataFrame(index=pd.date_range(start,periods=duration*4+1,freq="q").shift(n=-1,freq="m").shift(n=start.day,freq="D"))
        amortization["principal"]=0; amortization["remained_principal"]=0;amortization["interest"]=0;amortization["payback"]=0;
        amortization["remained_principal"][0]=principal;
        if(term=="y"):
            amortization.loc[::4,"principal"]=params["principal"]/duration; amortization.principal[0]=0;
        if(term=="q"):
            amortization["principal"]=principal/(duration*4);amortization.principal[0]=0
        for i in range(1, amortization.shape[0]):
            amortization.remained_principal.iloc[i]=amortization.remained_principal.iloc[i-1]-amortization.principal.iloc[i];
            amortization.interest[i]=amortization.remained_principal[i-1]*interest/4
    if(interest_term=="m"):
        amortization=pd.DataFrame(index=pd.date_range(start,periods=duration*12+1,freq="m").shift(n=-1,freq="m").shift(n=start.day,freq="D"))
        amortization["principal"]=0; amortization["remained_principal"]=0;amortization["interest"]=0;amortization["payback"]=0;
        amortization["remained_principal"][0]=principal;
        if(term=="y"):
            amortization.loc[::12,"principal"]=params["principal"]/duration; amortization.principal[0]=0;
        if(term=="q"):
            amortization.loc[::3,"principal"]=principal/(duration*4);amortization.principal[0]=0;
        if(term=="m"):
            amortization["principal"]=principal/(duration*12); amortization.principal[0]=0;
        for i in range(1, amortization.shape[0]):
            amortization.remained_principal.iloc[i]=amortization.remained_principal[i-1]-amortization.principal.iloc[i];
            amortization.interest[i]=amortization.remained_principal[i-1]*interest/12
    amortization["payback"]=amortization["principal"]+amortization["interest"]
    amortization=amortization.apply(money_trim,axis=0)
    return amortization

#원리금 균등상환(Constant Payment Mortgage)
def CPM_calc(startdate,duration,principal,interest,term):
    start=datetime.strptime(startdate,'%Y-%m-%d')
    if(term=="y"):
        amortization=pd.DataFrame(index=pd.date_range(start,periods=duration+1,freq="Y").shift(n=start.day,freq="D"))
        amortization["payback"]=0;amortization["interest"]=0;amortization["principal"]=0
        amortization.principal.iloc[1:amortization.shape[0]]=-np.ppmt(params["interest"], amortization.principal.iloc[1:amortization.shape[0]],params["duration"],params["principal"])
        amortization.interest.iloc[1:amortization.shape[0]]=-np.ipmt(params["interest"], amortization.interest.iloc[1:amortization.shape[0]],params["duration"],params["principal"])
        amortization.payback.iloc[1:amortization.shape[0]]=-np.pmt(interest, duration,principal)
        amortization.payback=amortization.apply(money_trim, axis=1)
    if(term=="q"):
        amortization=pd.DataFrame(index=pd.date_range(start,periods=duration*4+1,freq="q").shift(n=start.day,freq="D"))
        amortization["payback"]=0;amortization["interest"]=0;amortization["principal"]=0
        amortization.principal.iloc[1:amortization.shape[0]]=-np.ppmt(params["interest"]/4, amortization.principal.iloc[1:amortization.shape[0]],params["duration"]*4,params["principal"])
        amortization.interest.iloc[1:amortization.shape[0]]=-np.ipmt(params["interest"]/4, amortization.interest.iloc[1:amortization.shape[0]],params["duration"]*4,params["principal"])
        amortization.payback.iloc[1:amortization.shape[0]]=-np.pmt(interest/4, duration*4,principal)
        amortization.payback=amortization.apply(money_trim, axis=1)
    if(term=="m"):
        amortization=pd.DataFrame(index=pd.date_range(start,periods=duration*12+1,freq="M").shift(n=start.day,freq="D"))
        amortization["payback"]=0;amortization["interest"]=0;amortization["principal"]=0
        amortization.principal.iloc[1:amortization.shape[0]]=-np.ppmt(params["interest"]/12, amortization.principal.iloc[1:amortization.shape[0]],params["duration"]*12,params["principal"])
        amortization.interest.iloc[1:amortization.shape[0]]=-np.ipmt(params["interest"]/12, amortization.interest.iloc[1:amortization.shape[0]],params["duration"]*12,params["principal"])
        amortization.payback.iloc[1:amortization.shape[0]]=-np.pmt(interest/12, duration*12,principal)
        amortization.payback=amortization.apply(money_trim, axis=1)
    amortization=amortization.apply(money_trim,axis=0)
    return amortization

In [16]:
CAM_calc(params["startdate"],params["duration"],params["principal"],params["interest"],params["repayment_term"],params["interest_repayment_term"])

/Users/youngji/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/youngji/.local/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/youngji/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,principal,remained_principal,interest,payback
2018-05-10,0,120000000,0,0
2019-05-10,10000000,110000000,6000000,16000000
2020-05-10,10000000,100000000,5500000,15500000
2021-05-10,10000000,90000000,5000000,15000000
2022-05-10,10000000,80000000,4500000,14500000
2023-05-10,10000000,70000000,4000000,14000000
2024-05-10,10000000,60000000,3500000,13500000
2025-05-10,10000000,50000000,3000000,13000000
2026-05-10,10000000,40000000,2500000,12500000
2027-05-10,10000000,30000000,2000000,12000000


2460950